In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor
from tqdm.contrib.itertools import product

In [ ]:
df = pd.read_parquet("normalised_data.parquet")

In [ ]:
df_ = df['number_trips']
df_ = df_.rolling(window=24).mean()

In [ ]:
df_ = df_.dropna().reset_index()

In [ ]:
df_ = df_.drop(['index'], axis=1)

In [ ]:
test = df_[-24*30:]

In [ ]:
train = df_[:-24*30]

In [ ]:
model = ARIMA(train, order=(15,1,1),)

In [ ]:
model_fit = model.fit()

In [ ]:
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:           number_trips   No. Observations:               112397
Model:                ARIMA(15, 1, 1)   Log Likelihood              608243.031
Date:                Fri, 12 Apr 2024   AIC                       -1216452.061
Time:                        17:42:17   BIC                       -1216288.355
Sample:                             0   HQIC                      -1216402.640
                             - 112397                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.8826      0.001   1003.537      0.000       0.881       0.884
ar.L2          0.1776      0.001    289.532      0.000       0.176       0.179
ar.L3         -0.2788      0.002   -123.113      0.000      -0.283      -0.274
ar.L4         -0.0205      0.002     -8.338      0.000      -0.025      -0.016
ar.L5         -0.0547      0.003    -21.878      0.000      -0.060      -0.050
ar.L6          0.0507      0.003     19.969      0.000       0.046       0.056
ar.L7          0.0006      0.003      0.201      0.841      -0.005       0.006
ar.L8         -0.0689      0.003    -23.730      0.000      -0.075      -0.063
ar.L9          0.0359      0.003     12.302      0.000       0.030       0.042
ar.L10         0.1258      0.003     46.156      0.000       0.120       0.131
ar.L11        -0.0214      0.003     -6.852      0.000      -0.028      -0.015
ar.L12        -0.0675      0.003    -21.572      0.000      -0.074      -0.061
ar.L13         0.0089      0.003      2.814      0.005       0.003       0.015
ar.L14         0.0358      0.003     11.802      0.000       0.030       0.042
ar.L15        -0.0154      0.002     -6.704      0.000      -0.020      -0.011
ma.L1          0.4585      0.001    632.914      0.000       0.457       0.460
sigma2      1.164e-06   1.39e-09    839.392      0.000    1.16e-06    1.17e-06
===================================================================================
Ljung-Box (L1) (Q):                   0.14   Jarque-Bera (JB):          13416149.09
Prob(Q):                              0.71   Prob(JB):                         0.00
Heteroskedasticity (H):               0.07   Skew:                            -0.45
Prob(H) (two-sided):                  0.00   Kurtosis:                        56.52
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 1.07e+17. Standard errors may be unstable.
"""

In [ ]:
pred = model_fit.forecast(steps=24*30)

In [ ]:
test_y = test['number_trips']

In [ ]:
mean_absolute_error(test_y, pred)

0.01148803271326231

In [ ]:
with open('arima_model.pkl', 'wb') as f:
    pickle.dump(model_fit, f)

In [ ]:
data_seasonal = np.array(df_)

In [ ]:
data_seasonal_2 = np.array(df['number_trips'][23:])

In [ ]:
res = np.zeros_like(data_seasonal)

In [ ]:
for i in range(len(data_seasonal)):
    res[i] = data_seasonal_2[i] - data_seasonal[i]

In [ ]:
res = pd.DataFrame(res)

In [ ]:
df = df[23:]

In [ ]:
res['time'] = df['time']

In [ ]:
res['month'] = res['time'].dt.month
res['day'] = res['time'].dt.day_of_week
res['hour'] = res['time'].dt.hour
res['date'] = res['time'].dt.day

In [ ]:
data = res.drop(['time'], axis=1)

In [ ]:
data.dropna(inplace=True)

In [ ]:
Y = data[0]
X = data.drop([0], axis=1)

In [ ]:
test_Y = Y[-30*24:]
Y = Y[:-30*24]
test_X = X[-30*24:]
X = X[:-30*24]

In [ ]:
train_X, valid_X, train_y, valid_y = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
min_loss = np.inf
for n, d in product(range(5,101,5), range(2,20)):
    clk = RandomForestRegressor(n_estimators=n, max_depth=d).fit(train_X, train_y)
    pred_y = clk.predict(valid_X)
    if min_loss > mean_absolute_error(valid_y, pred_y):
        min_loss = mean_absolute_error(valid_y, pred_y)
        best_n = n
        best_d = d
print(min_loss, best_n, best_d)

  0%|          | 0/360 [00:00<?, ?it/s]

0.04338665796187407 40 10


In [ ]:
clk = RandomForestRegressor(n_estimators=90, max_depth=10).fit(X, Y)

In [ ]:
pred_seasonal = clk.predict(test_X)

In [ ]:
with open('arima_model.pkl','rb') as f:
    model = pickle.load(f)

In [ ]:
pred_trend = model.forecast(steps=24*30)

In [ ]:
pred_Y = pred_seasonal+pred_trend

In [ ]:
test_Y = df[-24*30:]['number_trips']

In [ ]:
print(mean_absolute_error(test_Y,pred_Y))

0.04918739380698144


In [ ]:
with open('seasonal_model.pkl','wb') as f:
    pickle.dump(clk,f)

In [ ]:
X['month_sin'] = np.sin(X['month'] * (2. * np.pi / 12))
X['month_cos'] = np.cos(X['month'] * (2. * np.pi / 12))
X['day_sin'] = np.sin(X['day'] * (2. * np.pi / 7))
X['day_cos'] = np.cos(X['day'] * (2. * np.pi / 7))
X['hour_sin'] = np.sin(X['hour'] * (2. * np.pi / 24))
X['hour_cos'] = np.cos(X['hour'] * (2. * np.pi / 24))
X['date_sin'] = np.sin(X['date'] * (2. * np.pi / 31))
X['date_cos'] = np.cos(X['date'] * (2. * np.pi / 31))
test_X['month_sin'] = np.sin(test_X['month'] * (2. * np.pi / 12))
test_X['month_cos'] = np.cos(test_X['month'] * (2. * np.pi / 12))
test_X['day_sin'] = np.sin(test_X['day'] * (2. * np.pi / 7))
test_X['day_cos'] = np.cos(test_X['day'] * (2. * np.pi / 7))
test_X['hour_sin'] = np.sin(test_X['hour'] * (2. * np.pi / 24))
test_X['hour_cos'] = np.cos(test_X['hour'] * (2. * np.pi / 24))
test_X['date_sin'] = np.sin(test_X['date'] * (2. * np.pi / 31))
test_X['date_cos'] = np.cos(test_X['date'] * (2. * np.pi / 31))
X_ = X.drop(['month', 'day', 'hour', 'date'], axis=1)
test_X_ = test_X.drop(['month', 'day', 'hour', 'date'], axis=1)

In [ ]:
X_['seasonal_residuals'] = Y
test_X_['seasonal_residuals'] = test_Y

In [ ]:
X_.to_parquet('DL_X.parquet', index=False)
test_X_.to_parquet('DL_test_X.parquet', index=False)